In [1]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOHumanoidDR-v5" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
REWARD_DIM = 2 # CHANGE THIS TO THE NUMBER OF OBJECTIVES IN THE ENVIRONMENT
SEEDS = [5,26,47,76,92] # CHANGE THIS TO THE SEEDS YOU USE

### Get Normalization Params

In [2]:
from evaluation import get_eval_params

normalization_data = get_eval_params(ENV_NAME)['normalization']
normalization_data

{'MOHumanoidDefault-v5': {0: [0, 411], 1: [-40, 188]},
 'MOHumanoidLight-v5': {0: [0, 548], 1: [0, 151]},
 'MOHumanoidHeavy-v5': {0: [0, 232], 1: [0, 136]},
 'MOHumanoidLowDamping-v5': {0: [0, 374], 1: [0, 139]},
 'MOHumanoidHard-v5': {0: [0, 328], 1: [-10, 116]}}

In [3]:
best_front_paths = get_eval_params(ENV_NAME)['best_specialist_front_path']
best_front_paths

{'MOHumanoidDefault-v5': 'data/single_env/MOHumanoidDR-v5/MOHumanoidDefault-v5/MORL-D(MOSAC)-SB+PSA_seed5.csv',
 'MOHumanoidLight-v5': 'data/single_env/MOHumanoidDR-v5/MOHumanoidLight-v5/MORL-D(MOSAC)-SB+PSA_seed5.csv',
 'MOHumanoidHeavy-v5': 'data/single_env/MOHumanoidDR-v5/MOHumanoidHeavy-v5/MORL-D(MOSAC)-SB+PSA_seed5.csv',
 'MOHumanoidLowDamping-v5': 'data/single_env/MOHumanoidDR-v5/MOHumanoidLowDamping-v5/MORL-D(MOSAC)-SB+PSA_seed5.csv',
 'MOHumanoidHard-v5': 'data/single_env/MOHumanoidDR-v5/MOHumanoidHard-v5/MORL-D(MOSAC)-SB+PSA_seed5.csv'}

### Normalize Front and Calculate Normalized Hypervolume and EUM for Generalist and Specialist

Import helpers

In [4]:
import numpy as np

sys.path.append(os.path.abspath(os.path.join('../..')))

def get_normalized_vec_returns(all_vec_returns, minmax_range):
    minmax_array = np.array([minmax_range[i] for i in range(all_vec_returns.shape[-1])])
    min_vals = minmax_array[:, 0].reshape(1, 1, -1) # reshape to (1, 1, n_objectives) for broadcasting
    max_vals = minmax_array[:, 1].reshape(1, 1, -1)

    clipped_vec_returns = np.clip(all_vec_returns, min_vals, max_vals) # broadcasted clipping
    
    # Normalize
    normalized_vec_returns = (clipped_vec_returns - min_vals) / (max_vals - min_vals)
    
    return normalized_vec_returns

In [5]:
from mo_utils.performance_indicators import hypervolume, expected_utility
from mo_utils.weights import equally_spaced_weights

NUM_WEIGHTS = 100
EVAL_WEIGHTS = equally_spaced_weights(REWARD_DIM, NUM_WEIGHTS) 

EVAL_WEIGHTS[0:5]

[array([0., 1.]),
 array([0.00987073, 0.99012927]),
 array([0.01986059, 0.98013941]),
 array([0.02987081, 0.97012919]),
 array([0.03986195, 0.96013805])]

### Normalize the fronts and calculate normalized hypervolume and EUM for SPECIALIST

In [6]:
from helpers.utils import ENVIRONMENTS_MAP, ALGORITHMS

FRONT = "eval/discounted_front" # don't change this, front extracted for specialists are only the discounted ones!!
file_path = f"data/{FRONT}/{ENV_NAME}"
scores_save_path = f"data/scores/{ENV_NAME}"

os.makedirs(f"{scores_save_path}", exist_ok=True)

In [7]:
normalized_specialist_hypervolumes = []
normalized_specialist_eums = []

for env in ENVIRONMENTS_MAP[ENV_NAME]:
    min_max_ranges = normalization_data[env]
    best_env_front_path = best_front_paths[env]
    assert os.path.exists(best_env_front_path), f"File {best_env_front_path} does not exist"
    
    best_env_front = pd.read_csv(best_env_front_path)
    data_array = best_env_front.to_numpy()
    normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)
    normalized_specialist_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
    normalized_specialist_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

specialist_data = {f"normalized_hypervolume/{env}": [normalized_specialist_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
specialist_data.update({f"normalized_eum/{env}": [normalized_specialist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
specialist_normalized_hv = pd.DataFrame(specialist_data)
specialist_normalized_hv.to_csv(f"{scores_save_path}/specialist.csv", index=False)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False



### Normalize the fronts and calculate normalized hypervolume and EUM for GENERALIST

In [8]:
# Load the data
for algo in ALGORITHMS:
    for seed in SEEDS:
        normalized_hypervolumes = []
        normalized_eums = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            min_max_ranges = normalization_data[env]
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()
            normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)

            normalized_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
            normalized_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

        data = {f"normalized_hypervolume/{env}": [normalized_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        data.update({f"normalized_eum/{env}": [normalized_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
        df = pd.DataFrame(data)
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)
            

# Calculate NHGR

In [9]:
# get the normalized hypervolumes of the specialists
specialist_normalized_hv_data = pd.read_csv(f"{scores_save_path}/specialist.csv")

for algo in ALGORITHMS:
    for seed in SEEDS:
        nhgrs = []
        # get the normalized hypervolumes we extracted earlier
        file = f"{scores_save_path}/{algo}/seed_{seed}.csv"
        seed_normalized_hv_data = pd.read_csv(file)

        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            # Filter columns that start with "normalized_hypervolume"
            hv_columns = seed_normalized_hv_data.filter(like="normalized_hypervolume").columns

            for col in hv_columns:
                specialist_normalized_hv = specialist_normalized_hv_data[col].values[0]
                generalist_normalized_hv = seed_normalized_hv_data[col].values[0]
                
                env_nhgr = generalist_normalized_hv / specialist_normalized_hv

                seed_normalized_hv_data[f'NHGR/{env}'] = env_nhgr
        
        seed_normalized_hv_data.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)